In [1]:
from modules.helper import *
import modules.lyapunov as lyp
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

tolerence = 1e-1
imagefolder = 'images/'

import modules.main as main
import time

start = time.time()
l1,l2 = main.main(amin = 0.7, amax = 2, lmin = 0.1, lmax = 1, da = 0.01, dl =0.01, ablocks = 50, lblocks = 50, n_transient = 1e6, n_attractor = 1e4)
print(f'Total run time:\t{time.time()-start:.2f}')

Iterating over the transient of 1000000 steps
    Done in 148.86 s
Iterating over the attractor of 10000 steps
    Done in 1.57 s
Generating initial conditions
    Done in 0.00 s
Calculating the Jacobian Matrix for each point
    Done in 0.68 s
System set up for analysis
Calculating the Lyapunov Exponents
0% done iterating in 0.00s
10% done iterating in 0.11s
20% done iterating in 0.11s
30% done iterating in 0.12s
40% done iterating in 0.11s
50% done iterating in 0.11s
60% done iterating in 0.11s
70% done iterating in 0.11s
80% done iterating in 0.12s
90% done iterating in 0.11s
This took 1.18 seconds to run
1/2500 in 152.30
Iterating over the transient of 1000000 steps


In [ ]:
l1.name = 'l1'
l2.name = 'l2'
plt.figure()
plt.title('Mean first Lyapunov Exponent')
l1.mean(axis=(0,1)).plot()
plt.savefig(imagefolder + 'Mean_lyapunov1.pdf')
plt.show()

plt.figure()
l2.mean(axis=(0,1)).plot()
plt.savefig(imagefolder + 'Mean_lyapunov_sum_12.pdf')
plt.show()

plt.figure()
(l2-l1).mean(axis=(0,1)).plot()
plt.show()

In [ ]:
l1.to_netcdf('data/lyapunov1_large_6_4_highres.nc')
l2.to_netcdf('data/lyapunov2_large_6_4_highres.nc')

In [ ]:
l1 = xr.open_dataset('data/lyapunov1.nc').l1
l2 = xr.open_dataset('data/lyapunov2.nc').l2
l1, l2

In [ ]:
stacked1 = l1.stack(z=('x','y','l','a'))
stacked2 = l2.stack(z=('x','y','l','a'))
stacked1 = stacked1.to_pandas()
stacked1 = stacked1.reset_index()
stacked1.columns = ['x_0', 'y_0', 'lambda', 'a', 'lyapunov1']
stacked1['lyapunov_sum_12'] = stacked2.values
stacked1.to_csv('data/full_data.dat', index = False)